# Initial Exploratory Taxi Trip DataSet
The objective of this notebook is to carry out an initial analysis of the taxi trips in Chicago and then on the basis of this analysis carry out a filtering of the erroneous trips.

## Chicago Coordinates 
-87.6244212, 41.8755616

## 1 Create our environment

### 1.1 Create the Spark Session

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext

In [3]:
sc_conf = SparkConf()

In [4]:
sc_conf.set('spark.driver.port', '62678')
sc_conf.set('spark.rdd.compress', 'True')
sc_conf.set('spark.driver.host', '127.0.0.1')
sc_conf.set('spark.serializer.objectStreamReset', '100')
sc_conf.set('spark.master', 'local[*]')
sc_conf.set('spark.executor.id', 'driver')
sc_conf.set('spark.submit.deployMode', 'client')
sc_conf.set('spark.ui.showConsoleProgress', 'true')
sc_conf.set('spark.app.name', 'pyspark-shell')
sc_conf.set("spark.executor.memory","6g")
sc_conf.set("spark.driver.memory","1g")

In [5]:
sc_conf.getAll()

dict_items([('spark.driver.port', '62678'), ('spark.rdd.compress', 'True'), ('spark.driver.host', '127.0.0.1'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.executor.id', 'driver'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true'), ('spark.app.name', 'pyspark-shell'), ('spark.executor.memory', '6g'), ('spark.driver.memory', '1g')])

In [6]:
sc = SparkContext(conf=sc_conf)

In [7]:
sql = SQLContext(sc)

In [8]:
session = sql.sparkSession
session

In [9]:
session.sparkContext.getConf().getAll()

[('spark.executor.memory', '6g'),
 ('spark.driver.port', '62678'),
 ('spark.driver.host', '127.0.0.1'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.driver.memory', '1g'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.app.id', 'local-1556728644481')]

## 2 Load the libraries

In [10]:
%matplotlib inline

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql import types as T
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = None

## 3 Read the data

In [11]:
taxi_schema = StructType([
    StructField("Trip ID", StringType(), nullable = True),
    StructField("Taxi ID", StringType(), nullable = True),
    StructField("Trip Start Timestamp", StringType(), nullable = True),
    StructField("Trip End Timestamp", StringType(), nullable = True),
    StructField("Trip Seconds", IntegerType(), nullable = True),
    StructField("trip_miles", DoubleType(), nullable = True),
    StructField("Pickup Census Tract", LongType(), nullable = True),
    StructField("Dropoff Census Tract", LongType(), nullable = True),
    StructField("Pickup Community Area", IntegerType(), nullable = True),
    StructField("Dropoff Community Area", IntegerType(), nullable = True),
    StructField("Fare", DoubleType(), nullable = True),
    StructField("Tips", DoubleType(), nullable = True),
    StructField("Tolls", DoubleType(), nullable = True),
    StructField("Extras", DoubleType(), nullable = True),
    StructField("Trip Total", DoubleType(), nullable = True),
    StructField("Payment Type", StringType(), nullable = True),
    StructField("Company", StringType(), nullable = True),
    StructField("Pickup Centroid Latitude", DoubleType(), nullable = True),
    StructField("Pickup Centroid Longitude", DoubleType(), nullable = True),
    StructField("Pickup Centroid Location", StringType(), nullable = True),
    StructField("Dropoff Centroid Latitude", DoubleType(), nullable = True),
    StructField("Dropoff Centroid Longitude", DoubleType(), nullable = True),
    StructField("Dropoff Centroid  Location", StringType(), nullable = True),
    StructField("Community Areas", IntegerType(), nullable = True)])

In [12]:
taxi_df = session.read.csv('../Data/Taxi_Trips.csv',
                              header=True,
                              schema = taxi_schema)
taxi_df.show(5)

+--------------------+--------------------+--------------------+--------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+----+-----+------+----------+------------+--------------------+------------------------+-------------------------+------------------------+-------------------------+--------------------------+--------------------------+---------------+
|             Trip ID|             Taxi ID|Trip Start Timestamp|  Trip End Timestamp|Trip Seconds|trip_miles|Pickup Census Tract|Dropoff Census Tract|Pickup Community Area|Dropoff Community Area|Fare|Tips|Tolls|Extras|Trip Total|Payment Type|             Company|Pickup Centroid Latitude|Pickup Centroid Longitude|Pickup Centroid Location|Dropoff Centroid Latitude|Dropoff Centroid Longitude|Dropoff Centroid  Location|Community Areas|
+--------------------+--------------------+--------------------+--------------------+------------+----------+-------------------

## 4 Initial Data analysis 

### 4.1 Get some trips to check the data

In [13]:
taxi_df.limit(5).toPandas()

,Trip ID,Taxi ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,trip_miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,Community Areas
0,2014a7f89716c3cccc7d7935a3dfd16d8908f33c,9492b268e840fcd19b554ae0d61ab86a48eee56b7fba98...,09/22/2015 05:30:00 PM,09/22/2015 05:45:00 PM,420,1.1,1.703184e+10,1.703108e+10,32,8,6.25,0.0,0.0,0.0,6.25,Cash,None,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.900221,-87.629105,POINT (-87.6291051864 41.9002212967),38
1,2014a81cdb5b71886837612c5e53d59c72f11b2d,d552bd52346c36f10ea828d1f770cf358278f7beecf08a...,09/17/2013 06:45:00 PM,09/17/2013 07:00:00 PM,540,3.2,NaN,NaN,8,24,9.65,0.0,0.0,0.0,9.65,Cash,None,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.901207,-87.676356,POINT (-87.6763559892 41.9012069941),37
2,2014a8218f8d69bb62b838f31f3f59bf98ff9478,1df52fc60f8edec541c9a014b8ecf83c86f471b328635c...,01/02/2016 10:30:00 PM,01/02/2016 10:30:00 PM,180,0.8,1.703108e+10,1.703108e+10,8,8,5.25,0.0,0.0,0.0,5.25,Cash,None,41.892042,-87.631864,POINT (-87.6318639497 41.8920421365),41.900266,-87.632109,POINT (-87.6321092196 41.9002656868),37
3,2014a86b8e1e98a8647e0dc815cac0537ab5c40e,d96279b9f0d9e121dca78282e8ead29d8a7aa18dc89d78...,12/15/2015 11:45:00 AM,12/15/2015 11:45:00 AM,360,0.0,NaN,NaN,7,7,6.05,0.0,0.0,0.0,6.05,Cash,Taxi Affiliation Services,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),68
4,2014a8724a96df31e968f45925101c31d2e6ea31,a485350f36958a2783d154ee7562f7809254058189b91d...,06/03/2013 07:00:00 AM,06/03/2013 07:15:00 AM,480,0.9,1.703184e+10,1.703132e+10,32,32,6.45,0.0,0.0,0.0,6.45,Cash,None,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),38


The following columns can be deleted because they do not add value as a result of which they are unions or repetitions of other columns:

    - Pickup Census Tract	
    - Dropoff Census Tract
    - Pickup Centroid Location
    - Dropoff Centroid Location
    - Community Areas


### 4.2 Study the dimensions of the dataset

In [14]:
taxi_df.count()

112860054

In [15]:
len(taxi_df.columns)

24

### 4.3 Check the names and types of the variables

In [16]:
taxi_df.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

### 4.4 Change the name of the variables to comply with good practices

In [17]:
for col in taxi_df.columns:
    new_col=col.lower().replace(" ","_")
    taxi_df = taxi_df.withColumnRenamed(col,new_col)
taxi_df.printSchema()

root
 |-- trip_id: string (nullable = true)
 |-- taxi_id: string (nullable = true)
 |-- trip_start_timestamp: string (nullable = true)
 |-- trip_end_timestamp: string (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- pickup_census_tract: long (nullable = true)
 |-- dropoff_census_tract: long (nullable = true)
 |-- pickup_community_area: integer (nullable = true)
 |-- dropoff_community_area: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- extras: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- company: string (nullable = true)
 |-- pickup_centroid_latitude: double (nullable = true)
 |-- pickup_centroid_longitude: double (nullable = true)
 |-- pickup_centroid_location: string (nullable = true)
 |-- dropoff_centroid_latitude: double (nullable = true)
 |-- dropoff_centroid_

### 4.5 Delete fields that do not add value

In [18]:
#The following columns can be deleted because they do not add value as a result of which they are unions or 
#repetitions of other columns
taxi_df = taxi_df.drop('pickup_census_tract',
                       'dropoff_census_tract',
                       'pickup_centroid_location',
                       'dropoff_centroid__location',
                       'community_areas')

In [19]:
taxi_df.printSchema()

root
 |-- trip_id: string (nullable = true)
 |-- taxi_id: string (nullable = true)
 |-- trip_start_timestamp: string (nullable = true)
 |-- trip_end_timestamp: string (nullable = true)
 |-- trip_seconds: integer (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- pickup_community_area: integer (nullable = true)
 |-- dropoff_community_area: integer (nullable = true)
 |-- fare: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- extras: double (nullable = true)
 |-- trip_total: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- company: string (nullable = true)
 |-- pickup_centroid_latitude: double (nullable = true)
 |-- pickup_centroid_longitude: double (nullable = true)
 |-- dropoff_centroid_latitude: double (nullable = true)
 |-- dropoff_centroid_longitude: double (nullable = true)



In [20]:
# We save it in a temporal folder 
taxi_df.write.parquet("../Data/temp/taxi_trips_initial.parquet")
taxi_df = session.read.parquet("../Data/temp/taxi_trips_initial.parquet")

### 4.6 Manage the dates

#### 4.6.1 Convert the dates from string format to date format

In [ ]:
# Check the start trip and the end trip before change the format
taxi_df.select(['trip_start_timestamp', 'trip_end_timestamp']).show(truncate=False)

In [ ]:
#Convert to date format ()
taxi_df = taxi_df.withColumn("trip_start_timestamp",
                             F.from_unixtime(F.unix_timestamp(F.col("trip_start_timestamp"),
                                                           format="MM/dd/yyyy hh:mm:ss aa")))
taxi_df = taxi_df.withColumn("trip_end_timestamp",
                             F.from_unixtime(F.unix_timestamp(F.col("trip_end_timestamp"),
                                                           format="MM/dd/yyyy hh:mm:ss aa")))

In [ ]:
#Check the result
taxi_df.select(['trip_start_timestamp', 'trip_end_timestamp']).show(truncate=False)

#### 4.6.2 Check the extreme values of the dates

In [ ]:
taxi_df.select(F.max('trip_start_timestamp'),F.min('trip_start_timestamp')).show()

In [ ]:
taxi_df.select(F.max('trip_end_timestamp'),F.min('trip_end_timestamp')).show()

### 4.7 Study if there are variables that always or practically always have the same value

Separate the dataframe in two to realize our analysis:
    
    - One dataframe with numeric columns
    - One dataframe with categorical columns

In [22]:
n_rows = taxi_df.count()
numeric_vars = []
categoric_vars = []

for col, tipo in taxi_df.dtypes:
    if tipo!="string":
        numeric_vars.append(col)
    else:
        categoric_vars.append(col)     

In [ ]:
# We delete trip_id from our analysis because it is the only value that allows us to identify travel and we know it´s
# an unique value (it has not duplicates)
categoric_vars.remove('trip_id')

#### 4.7.1 Categoric columns

In [ ]:
for col in categoric_vars:
    print(col)
    taxi_df.groupby(col).count().sort(F.col("count").desc()).withColumn("frecuencia(%)",
                  F.round((100*F.col("count")/n_rows),2)).show()

We have observed that in the column 'payment_type' there is the value 'Unknown', so we are going to convert it to null

In [ ]:
taxi_df = taxi_df.withColumn('payment_type',
                            F.when(F.col('payment_type')=='Unknown',None).otherwise(F.col('payment_type')))

In [ ]:
#Check that 'Unknown' value has been converted to null
print('payment_type')
taxi_df.groupby('payment_type').count().sort(F.col("count").desc()).withColumn("frecuencia(%)",
                                                                    F.round((100*F.col("count")/n_rows),2)).show()

#### 4.7.2 Numeric columns

In [ ]:
for col in numeric_vars:
    print(col)
    taxi_df.groupby(col).count().sort(F.col("count").desc()).withColumn("frecuencia(%)",
                  F.round((100*F.col("count")/n_rows),2)).show()

### 4.8 Study the null values 

### 4.8.1 Get the number of nulls

In [23]:
nulls = {}
for column in taxi_df.columns:
    n_nulls = taxi_df.filter(F.col(column).isNull()).count()
    perc_nulls = 100 * n_nulls / n_rows
    nulls[column]=[n_nulls,round(perc_nulls,2)]

nulls_df = pd.DataFrame(nulls).T
nulls_df.columns = ['nulls','% nulls']
nulls_df

,nulls,% nulls
trip_id,0.0,0.00
taxi_id,0.0,0.00
trip_start_timestamp,0.0,0.00
trip_end_timestamp,15868.0,0.01
trip_seconds,1289707.0,1.14
trip_miles,633.0,0.00
pickup_community_area,16782342.0,14.87
dropoff_community_area,19095838.0,16.92
fare,1076.0,0.00
tips,1076.0,0.00


### 4.8.2 Plot the number of nulls

In [ ]:
nulls_df.sort_values('% nulls').plot.barh(y='% nulls',figsize=(10,10))

### 4.9 Check duplicates

In [ ]:
taxi_df.dropDuplicates(['trip_id', 'taxi_id']).count()

There is not duplicates

## 5 Clean the dataset

### 5.1 Drop the trips with null values

In [24]:
taxi_df = taxi_df.dropna(how='any',
                         subset=['trip_id',
                                  'taxi_id',
                                  'trip_start_timestamp',
                                  'trip_end_timestamp',
                                  'trip_seconds',
                                  'trip_miles',
                                  'pickup_community_area',
                                  'dropoff_community_area',
                                  'fare',
                                  'tips',
                                  'tolls',
                                  'extras',
                                  'trip_total',
                                  'payment_type',
                                  'company',
                                  'pickup_centroid_latitude',
                                  'pickup_centroid_longitude',
                                  'dropoff_centroid_latitude',
                                  'dropoff_centroid_longitude'])

In [25]:
taxi_df.count()

60535858

### 5.2 Drop the trips with extrange values

In [26]:
taxi_df = taxi_df.filter((F.col("trip_start_timestamp") <= (F.col("trip_end_timestamp"))) &
                (F.col("trip_seconds") > 60) &
                (F.col("trip_miles") > 0.5) &
                (F.col("fare") > 0) &
                (F.col("tips") >= 0) &
                (F.col("tolls") >= 0) &
                (F.col("extras") >= 0) &
                (F.col("trip_total") > 0))

In [27]:
taxi_df.count()

29210670

### 5.3 Check that all the null values have been deleted

In [28]:
n_rows = taxi_df.count()
nulls = {}
for column in taxi_df.columns:
    n_nulls = taxi_df.filter(F.col(column).isNull()).count()
    perc_nulls = 100 * n_nulls / n_rows
    nulls[column]=[n_nulls,round(perc_nulls,2)]

nulls_df = pd.DataFrame(nulls).T
nulls_df.columns = ['nulls','% nulls']
nulls_df

,nulls,% nulls
trip_id,0.0,0.0
taxi_id,0.0,0.0
trip_start_timestamp,0.0,0.0
trip_end_timestamp,0.0,0.0
trip_seconds,0.0,0.0
trip_miles,0.0,0.0
pickup_community_area,0.0,0.0
dropoff_community_area,0.0,0.0
fare,0.0,0.0
tips,0.0,0.0
